Add the example code to your application

1
Configure your user pool app client with allowed callback URLs, logout URLs, and the scopes that you want to request, for example openid and profile. Learn more 

2
Install the authlib  library.

In [ ]:
pip install authlib werkzeug flask requests

3
Add and configure the authlib OAuth component.

In [ ]:
from flask import Flask, redirect, url_for, session
from authlib.integrations.flask_client import OAuth
import os
 
app = Flask(__name__)
app.secret_key = os.urandom(24)  # Use a secure random key in production
oauth = OAuth(app)

oauth.register(
  name='oidc',
  authority='https://cognito-idp.us-east-2.amazonaws.com/us-east-2_eurel0TFX',
  client_id='638bt1ru33a9489bv8ai066kb4',
  client_secret='<client secret>',
  server_metadata_url='https://cognito-idp.us-east-2.amazonaws.com/us-east-2_eurel0TFX/.well-known/openid-configuration',
  client_kwargs={'scope': 'phone openid email'}
)

4
Add a home page with links to login and logout routes.

In [ ]:
@app.route('/')
def index():
    user = session.get('user')
    if user:
        return  f'Hello, {user["email"]}. <a href="/logout">Logout</a>'
    else:
        return f'Welcome! Please <a href="/login">Login</a>.'

5
Configure a login route to direct to Amazon Cognito managed login for authentication with a redirect to an authorize route.



In [ ]:
@app.route('/login')
def login():
    # Alternate option to redirect to /authorize
    # redirect_uri = url_for('authorize', _external=True)
    # return oauth.oidc.authorize_redirect(redirect_uri)
    return oauth.oidc.authorize_redirect('https://dashboard')

6
The OAuth module collects the access token and retrieves user data from the Amazon Cognito userInfo endpoint. Configure an authorize route to handle the access token and user data after authentication.

In [ ]:
@app.route('/authorize')
def authorize():
    token = oauth.oidc.authorize_access_token()
    user = token['userinfo']
    session['user'] = user
    return redirect(url_for('index'))

7
Configure a logout route that erases user session data.

In [ ]:
@app.route('/logout')
def logout():
    session.pop('user', None)
    return redirect(url_for('index'))

if __name__ == '__main__':
    app.run(debug=True)